In [15]:
from google.cloud import bigquery
import os
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd
import datetime
from google.cloud import bigquery
import pytz
from google.oauth2 import service_account
import numpy as np
import requests
from datetime import datetime as dt
from datetime import date


In [16]:
os.listdir()

['.ipynb_checkpoints',
 'boston_extract_code.ipynb',
 'chattanooga_extract_code.ipynb',
 'chicago_extract_code.ipynb',
 'creds',
 'kc_extract_code.ipynb',
 'littlerock_extract_code.ipynb',
 'nyc_extract_code.ipynb']

In [17]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = r'.\creds\opendatadbt-f2e674527eb6.json'
client = bigquery.Client()

credentials = ServiceAccountCredentials.from_json_keyfile_name(r'.\creds\opendatadbt-f2e674527eb6.json') # Your json file here



gc = gspread.authorize(credentials)

wks = gc.open("NYTimes Mini Crossword times").sheet1


data = wks.get_all_values()
headers = data.pop(0)

df = pd.DataFrame(data, columns=headers)


df = df.replace(np.NaN, -1)

# saving a data frame to a buffer (same as with a regular file):
df.to_csv('nytimescrossword.csv',index=False)

# NYC

#Initial Load
nyc_table_id = "opendatadbt.311.nyc311"
nyc311 = requests.get("https://data.cityofnewyork.us/resource/erm2-nwe9.json?$limit=10000&$where=created_date >= '2021-03-18T00:00:00.000' OR closed_date >= '2021-03-18T00:00:00.000'" )
nyc311json = nyc311.json()
nyc_df = pd.DataFrame(nyc311json)
#nyc_df = nyc_df[['created_date','closed_date','agency','agency_name','descriptor','incident_zip','incident_address','city','status','latitude','longitude']]

In [18]:
#Incremental load
nyc_table_id = "opendatadbt.311.nyc311"
current_datetime = dt.today().strftime("%Y-%m-%dT%H:%M:%S.000")
query_string = """
SELECT 
max(created_date), max(closed_date)
FROM `opendatadbt.311.nyc311`
limit 10
"""

nycmaxopendatedataframe = (
    client.query(query_string)
    .result()
    .to_dataframe()
)
nycmaxopendatedate = str(nycmaxopendatedataframe['f0_'][0])

nycmaxclosedatedataframe = (
    client.query(query_string)
    .result()
    .to_dataframe()
)
nycmaxclosedate = str(nycmaxclosedatedataframe['f1_'][0])


In [19]:

nyc311 = requests.get("https://data.cityofnewyork.us/resource/erm2-nwe9.json?$limit=50000&$where=created_date>'"+nycmaxopendatedate+"' OR closed_date between '" +nycmaxclosedate+ "' AND '" + current_datetime + "'") 
nyc311json = nyc311.json()
cols = ['unique_key','created_date','closed_date','agency','agency_name','descriptor','incident_zip','incident_address','city','status','latitude','longitude']
nyc_df = pd.DataFrame(nyc311json, columns=cols)
nyc_df

,unique_key,created_date,closed_date,agency,agency_name,descriptor,incident_zip,incident_address,city,status,latitude,longitude
0,50087168,2021-03-24T00:58:27.000,NaN,NYPD,New York City Police Department,Banging/Pounding,11374,99-65 64 ROAD,REGO PARK,In Progress,40.730831946852234,-73.8541380464189
1,50087296,2021-03-24T00:43:31.000,NaN,DOT,Department of Transportation,Stop,10469,2475 GUNTHER AVENUE,BRONX,In Progress,40.86203066169051,-73.83613559654096
2,50088494,2021-03-24T00:28:40.000,NaN,NYPD,New York City Police Department,Blocked Hydrant,11103,25-74 49 STREET,ASTORIA,In Progress,40.762900638851654,-73.90588737638393
3,50094896,2021-03-24T00:17:55.000,NaN,NYPD,New York City Police Department,Loud Music/Party,11207,722 CENTRAL AVENUE,BROOKLYN,In Progress,40.68677700620914,-73.90528448613152
4,50096013,2021-03-24T00:27:40.000,2021-03-24T01:02:01.000,NYPD,New York City Police Department,Car/Truck Music,10017,333 EAST 49 STREET,NEW YORK,Closed,40.75387699991748,-73.96807874069368
...,...,...,...,...,...,...,...,...,...,...,...,...
173,50096043,2021-03-24T00:53:09.000,NaN,NYPD,New York City Police Department,Car/Truck Music,10462,WATERBURY AVENUE,NaN,In Progress,40.83537424093395,-73.84218208377251
174,50094676,2021-03-24T00:25:05.000,2021-03-24T00:57:31.000,NYPD,New York City Police Department,Banging/Pounding,10468,2285 UNIVERSITY AVENUE,BRONX,Closed,40.86063552434622,-73.90685892732856
175,50094742,2021-03-24T00:51:14.000,2021-03-24T01:01:44.000,NYPD,New York City Police Department,N/A,10468,2272 UNIVERSITY AVENUE,BRONX,Closed,40.86040237374894,-73.90704362661752
176,50094918,2021-03-24T00:07:36.000,2021-03-24T00:52:20.000,NYPD,New York City Police Department,Loud Music/Party,11237,442 SUYDAM STREET,BROOKLYN,Closed,40.70664054277396,-73.9192182364529


In [20]:
nyc_table_id = "opendatadbt.311.nyc311"
dataset_ref = client.dataset('311')
table_ref = dataset_ref.table('nyc311')
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_TRUNCATE",
)

client.load_table_from_dataframe(
    nyc_df, table_ref, job_config=job_config
).result()  # Make an API request.
